In [1]:
import datetime
import gzip
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd

import tools

In [2]:
filename = '2015_07_22_mktplace_shop_web_log_sample.log'

###### Expects the unpacked log file to be in same directory, if not, then unpack from data/ directory


In [3]:
tools.extract_file(filename)

###### Read all lines into memory

In [4]:
with open(filename) as fin:
    lines = fin.readlines()

In [5]:
len(lines)

1158500

###### Match regex, parse each entry into dict, and perform quick sanity test

In [6]:
parsed_entries = [tools.parse_elb(l) for l in lines]

assert len(lines) == len(parsed_entries)
assert(l != None for l in lines)

##### Convert parsed entries into pandas DataFrame for analytics

In [7]:
batchdf = pd.DataFrame(parsed_entries, columns=tools.ELBHEADERS)
batchdf['timestamp'] = pd.to_datetime(batchdf['timestamp'])

# 1. Sessionize the web log by IP. Sessionize = aggregrate all page hits by visitor/IP during a session. https://en.wikipedia.org/wiki/Session_(web_analytics)

###### Approach:
###### 1. Sort by timestamp
###### 2. Group by client_ip and perform delta on timestamps
###### 3. For any row where timestamp delta >= 15 minutes, mark row as the first hit of a new session
###### 4. Create session_id based on format [ip]_[count]


In [8]:
session_df = tools.construct_sessions(batchdf)

In [9]:
session_df.head()

,timestamp,client_ip,diff,request_url,is_new_session,session_count,session_id
359679,2015-07-22 02:40:06.499174+00:00,106.51.235.133,00:00:00,https://paytm.com:443/shop/wallet/txnhistory?p...,0,0,106.51.235.133_0
359690,2015-07-22 02:40:06.632589+00:00,115.250.16.146,00:00:00,https://paytm.com:443/shop/wallet/balance?chan...,0,0,115.250.16.146_0
359676,2015-07-22 02:40:06.633226+00:00,106.51.235.133,00:00:00.134052,https://paytm.com:443/shop/wallet/balance?chan...,0,0,106.51.235.133_0
359677,2015-07-22 02:40:06.700160+00:00,52.74.219.71,00:00:00,https://paytm.com:443/papi/v1/promosearch/prod...,0,0,52.74.219.71_0
359678,2015-07-22 02:40:06.734226+00:00,27.97.124.172,00:00:00,https://paytm.com:443/coupons,0,0,27.97.124.172_0


# Q2: Determine the average session time

###### Approach:
###### 1. Group by session_id and perform cumsum() over timestamp diffs, thus the last entry of a session_id group is the total length of the session
###### 2. Collapse each session_id group to keep only the last entry
###### 3. Group by client_ip and perform mean(), std(), and sum() over the length of sessions per client_ip

In [10]:
session_df = tools.build_session_lengths(session_df)

In [11]:
statsdf = session_df[['client_ip','session_id','session_count','session_diff','session_length']]

In [12]:
merged = tools.build_stats(statsdf)

In [13]:
merged.head()

,client_ip,session_id,session_count,session_diff,session_length,mean,std,total
0,1.186.101.79,1.186.101.79_0,0,00:00:00.003948,00:18:37.542721,00:18:37.542721,NaT,00:18:37.542721
1,1.186.103.240,1.186.103.240_0,0,00:00:00.083478,00:00:11.611017,00:00:11.611017,NaT,00:00:11.611017
2,1.186.103.78,1.186.103.78_0,0,00:00:00.224246,00:00:09.000485,00:00:09.000485,NaT,00:00:09.000485
3,1.186.108.213,1.186.108.213_0,0,00:01:52.907065,00:01:52.907065,00:01:52.907065,NaT,00:01:52.907065
4,1.186.108.230,1.186.108.230_0,0,00:00:09.164021,00:00:18.815206,00:00:18.815206,NaT,00:00:18.815206


# Q3: Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session.

###### Approach:
###### 1. Group by session_id and count unique on request_url column

In [14]:
unique_url = session_df[['client_ip','request_url','session_id']]

In [ ]:
urldf = tools.build_url_stats(unique_url)

In [ ]:
urldf.head()

# Q4a: Find the most engaged users - Longest session times

###### Approach:
###### 1. Perform naive ranking on total length of sessions based on client_ip


In [ ]:
rankings = merged[['client_ip','session_count','total']]

In [ ]:
rankings.session_count+=1

In [ ]:
rankings = rankings.sort_values(['total'],ascending=[False])

In [ ]:
rankings.head()

# Q4b: Find the most engaged users - session count/mean/stddev based approach

###### Approach:
###### 1. Rank engagement based on session count, mean session times, stddev of session times

In [ ]:
rankings = merged[['client_ip','session_count','mean','std']]

In [ ]:
rankings.session_count+=1

In [ ]:
rankings = rankings.sort_values(['session_count','mean','std'], ascending=[False,False,True])

In [ ]:
rankings[rankings['client_ip'] == '122.248.163.2']

In [ ]:
rankings.head()